In [ ]:
import sys,tweepy,re 
from textblob import TextBlob
import matplotlib.pyplot as plt

class SentimentAnalysis:

    def init(self):
        self.tweets = []
        self.tweetText = []

    def downloadData(self):
        # authenticating
        consumerKey = 'QNWBxaEcMIgsZUITyL4H5WIqy'
        consumerSecret = '240Rl1ukNMyMth8z2js2uvkUGNyk6OMq4reRm0atuXuMc6PCK2'
        accessToken = '321231914-h4Rl0FfzgjbfRZAIarxIEMeDDQbpwTOXYZlifYHP'
        accessTokenSecret = 'VcJEsZCU81QHyIvhNpYXvT4JkfCsfzPV61p78x95HTVSX'

        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth)
        cnt = 0
        
        # input for term to be searched and how many tweets to search
        searchTerm = input("Enter Keyword to search about: ")
        noOfTerms = int(input("Enter Number of tweets to consider: "))

        # searching for tweets
        self.tweets = tweepy.Cursor(api.search_tweets,q=searchTerm, lang = "en").items(noOfTerms)
        # creating some variables to store info
        polarity = 0
        positive = 0
        spositive = 0
        negative = 0
        snegative = 0
        neutral = 0
        self.tweetText = []
        # iterating through tweets fetched
        for tweet in self.tweets:
            print("\n", cnt, ",tweet.text:", tweet.text)
            self.tweetText.append(self.cleanTweet(tweet.text).encode('utf-8'))
            analysis = TextBlob(tweet.text)
            polarity += analysis.sentiment.polarity  # adding up polarities to find the average 
            cnt=cnt+1
            if (analysis.sentiment.polarity == 0):  
                neutral += 1
            elif (analysis.sentiment.polarity > 0 and analysis.sentiment.polarity <= 0.5):
                positive += 1
            elif (analysis.sentiment.polarity > 0.5 and analysis.sentiment.polarity <= 1):
                spositive += 1
            elif (analysis.sentiment.polarity > -0.5 and analysis.sentiment.polarity <= 0):
                negative += 1
            elif (analysis.sentiment.polarity > -1 and analysis.sentiment.polarity <= -0.5):
                snegative += 1    
        if(cnt == 0):
            print("There is no tweets mentioning the given keyword.")
            return 0
        # finding average of reactions
        positive = self.percentage(positive, cnt)
        spositive = self.percentage(spositive, cnt)
        negative = self.percentage(negative, cnt)
        snegative = self.percentage(snegative, cnt)
        neutral = self.percentage(neutral, cnt)

        # finding average reaction
        polarity = polarity / cnt

        
        if (polarity == 0):
            senti = "Neutral"
        elif (polarity > 0 and polarity <= 0.5):
            senti = "Positive"
        elif (polarity > 0.5 and polarity <= 1):
            senti = "Strongly Positive"
        elif (polarity > -0.5 and polarity <= 0.0):
            senti = "Negative"
        elif (polarity > -1 and polarity <= -0.5):
            senti = "Strongly Negative"

            # printing the data
        print("General reaction of people on " + searchTerm + " by analyzing " + str(cnt) + " tweets is " + senti)

     
        self.plotPieChart(positive, spositive, negative, snegative, neutral, searchTerm, cnt)

    def cleanTweet(self, tweet):
        # Remove Links, Special Characters from tweet
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w +:\ / \ / \S +)", " ", tweet).split())

    # function to calculate percentage
    def percentage(self, part, whole):
        print("part:", part, ",whole:",whole)
        temp = 0
        if whole > 0:
            temp = 100 * float(part) / float(whole)
        return temp


    def plotPieChart(self, positive, spositive, negative, snegative, neutral, searchTerm, noOfSearchTerms):
        labels = ['Strongly Positive [' + str(spositive) + '%]','Positive [' + str(positive) + '%]', 'Neutral [' + str(neutral) + '%]',
                  'Negative [' + str(negative) + '%]', 'Strongly Negative [' + str(snegative) + '%]']
        sizes = [ spositive,positive, neutral, negative, snegative]
        colors = ['gold','yellow',  'green','red','darkred']
        patches, texts = plt.pie(sizes, colors=colors, startangle=0)
        plt.legend(patches, labels, loc="upper right")
        plt.title('How people are reacting on ' + searchTerm + ' by analyzing ' + str(noOfSearchTerms) + ' Tweets.')
        plt.axis('equal')
        plt.tight_layout()
        plt.show()


if __name__== "__main__":
    sa = SentimentAnalysis()
    sa.downloadData()